# Clinical Data Processing

## Where the data at?

In [1]:
input_path = '../Data/Intermediate_Files/'
clinicaldata_path = '../Data/Raw_Data/Clinical_Data/'
output_path = '../Data/Processed_Data/'

## Load Methyl Data

In [2]:
import pandas as pd

df_methyl = pd.read_pickle(
    input_path+'2_MethylData_Processing_Output.pkl')

# .T.reset_index(level=0, names='Batch')

print(
    f' Dataset (df) contains {df_methyl.shape[1]} columns (mC sites) and {df_methyl.shape[0]} rows (samples).')


 Dataset (df) contains 367395 columns (mC sites) and 3379 rows (samples).


In [3]:
df_methyl

IlmnID,Batch,cg00000029,cg00000109,cg00000165,cg00000236,cg00000292,cg00000321,cg00000363,cg00000622,cg00000658,...,ch.9.837340R,ch.9.84051654F,ch.9.84078312F,ch.9.86947500F,ch.9.88862796F,ch.9.898515R,ch.9.90287778F,ch.9.90621653R,ch.9.93402636R,ch.9.98463211R
202897270043_R01C01,GSE190931,NaN,0.932,0.101,0.838,0.893,0.225,0.389,0.013,0.905,...,0.047,0.061,0.050,0.055,0.051,0.077,0.068,0.043,0.036,0.043
202897270043_R03C01,GSE190931,0.720,0.962,0.588,0.822,0.899,0.549,0.543,0.014,0.862,...,0.074,0.028,0.064,0.045,0.036,0.045,0.027,0.027,0.025,0.030
202897270043_R04C01,GSE190931,0.867,0.881,0.157,0.904,0.736,0.860,0.673,0.018,0.797,...,0.074,0.035,0.151,0.046,0.091,NaN,0.053,0.051,0.044,0.044
202897270043_R05C01,GSE190931,NaN,0.876,0.625,NaN,0.219,0.360,0.154,0.019,0.866,...,NaN,0.039,0.217,0.060,0.071,NaN,0.066,0.048,0.059,0.122
202897270043_R06C01,GSE190931,0.575,0.924,NaN,0.674,0.683,0.295,0.188,0.015,0.861,...,0.085,0.053,0.034,0.045,0.076,NaN,0.038,0.037,0.035,0.054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203724130020_R02C01,GSE147667,0.875,0.939,0.167,0.944,0.936,0.291,0.336,0.024,0.848,...,0.064,0.039,0.030,0.050,0.044,0.073,0.058,0.040,0.029,0.059
203724130020_R03C01,GSE147667,0.882,0.930,0.132,0.942,0.945,0.331,0.382,0.024,0.855,...,0.036,0.035,0.035,0.052,0.050,0.086,0.055,0.036,0.034,0.044
203724130020_R04C01,GSE147667,0.854,0.934,0.131,0.940,0.941,0.337,0.368,0.022,0.867,...,0.042,0.038,0.047,0.053,0.039,0.075,0.051,0.040,0.034,0.041
203724130020_R05C01,GSE147667,0.863,0.923,0.146,0.943,0.955,0.347,0.350,0.023,0.864,...,0.069,0.046,0.041,0.048,0.039,0.077,0.044,0.058,0.041,0.044


## Add Labels/Clinical Outcome Data

In [4]:
# Import functions to clean up clinical data
from FM_Functions.Clinical_Data_CleanUp import *

# Combine all clinical data files into one dataframe and indexes it by the sample ID
labels_cog, labels_aml02, labels_aml08, labels_aml05 = combine_and_index_clinicaldata()
# Clean up and adjust clinical data labels
labels_aml02 = clean_aml02(labels_aml02)
labels_aml08 = clean_aml08(labels_aml08)
labels_cog   = clean_cog(labels_cog)
labels_aml05 = clean_aml05(labels_aml05)

# Combine all clinical data labels
df = pd.concat([labels_aml02, labels_aml08, labels_cog,
               labels_aml05], axis=0, join='outer')

# Remove samples that are not in the methyl dataset
# df = df.loc[df.index.isin(df_methyl.index)]

# Label control samples from the AML0531 clinical trial (GSE124413) as 'Bone Marrow Normal'

def label_control_samples(df_methyl, df):
    """
    This function labels control samples from the AML0531 clinical trial (GSE124413) as 'Bone Marrow Normal'
    and combines them with the clinical trial samples.
    """
    a = df_methyl[df_methyl['Batch'].isin(['GSE124413'])]
    b = df[df.index.isin(a.index)]
    control_0531 = a[~a.index.isin(b.index)]
    control_0531['Sample Type'] = 'Bone Marrow Normal'
    df_ = pd.concat(
        [df, control_0531['Sample Type'].to_frame()], axis=0, join='outer')
    return df_

df_ = label_control_samples(df_methyl, df)


In [21]:
df.shape

(2282, 291)

## BeatAML Clinical Data

In [ ]:
import pandas as pd
meta = pd.read_pickle('../Data/Raw_Data/GSE159907_BeatAML/GSE159907_GPL21145_meta_data.pkl')
# Create a new column with only the content inside [] from column 'Sample_Name'
meta['LLS_SampleID'] = meta['Sample_Name'].str.extract(r"\[(.*?)\]", expand=False)

# Set the index to the new column
meta1 = meta[['tissue','disease state','LLS_SampleID']].set_index('LLS_SampleID')

# Capitalize column names
meta1.columns = meta1.columns.str.capitalize()

# Read in the clinical data
meta2 = pd.read_excel('../Data/Raw_Data/Clinical_Data/BeatAML/BEAT_AML_Raw clinical data_702.Samples.Vizome.xlsx', index_col=3)

# Join the two dataframes
meta3 = meta1.join(meta2, how='left')

## Remove Samples based on Certain Clinical Features

### Remove Relapse Samples

In [5]:
df1 = df_[~df_['Sample Type'].isin(['Relapse', 'Recurrent Blood Derived Cancer - Bone Marrow',
                                    'Recurrent Blood Derived Cancer - Peripheral Blood'])]

print(
    f'Out of {df_.shape[0]} samples, {df_.shape[0]-df1.shape[0]} matched, yielding {df1.shape[0]} samples after filtering')


Out of 1762 samples, 248 matched, yielding 1514 samples after filtering


### Remove Control/Normal Samples

In [6]:
df2 = df1[~df1['Sample Type'].isin(
    ['Bone Marrow Normal', 'Blood Derived Normal'])]
print(
    f'Out of {df1.shape[0]} samples, {df1.shape[0]-df2.shape[0]} matched, yielding {df2.shape[0]} samples after filtering')


Out of 1514 samples, 154 matched, yielding 1360 samples after filtering


### Remove Duplicate Samples

In [7]:
df3 = df2[~df2['Patient_ID'].duplicated(keep='last')]
print(
    f'Out of {df2.shape[0]} samples, {df2.shape[0]-df3.shape[0]} matched, yielding {df3.shape[0]} samples after filtering')


Out of 1360 samples, 14 matched, yielding 1346 samples after filtering


## Save Files

In [8]:
output = df3.join(df_methyl,how='left') # Join clinical data with methyl data

x = output.iloc[:,df3.shape[1]+1:] # Select only methyl data
y = output.iloc[:,0:df3.shape[1]+1] # Select only clinical data

In [9]:
# Split train and test by clinical trial
y_train = y[~y['Clinical Trial'].isin(['AML02', 'AML08'])]
# y_train = y_train[y_train['Sample Type'].isin(['Diagnosis',
#        'Primary Blood Derived Cancer - Bone Marrow', 'Bone Marrow Normal',
#        'Primary Blood Derived Cancer - Peripheral Blood',
#        'Blood Derived Normal'])]

y_test = y[y['Clinical Trial'].isin(['AML02', 'AML08'])]

# Select samples in x that are in y_train
x_train = x.loc[y_train.index]
x_test = x.loc[y_test.index]

# x_train = pd.concat([x_train, ctrl_x], axis=0)
# y_train = pd.concat([y_train, ctrl_y], axis=0,keys=['Diagnosis','Control'], names=['sample_type'])


print(
    f"Discovery dataset (train) contains {x_train.shape[1]} rows (5mC sites) and {x_train.shape[0]} columns (samples)")
print(
    f"\n{y_train['Clinical Trial'].value_counts(dropna=False).to_string()}\n")
print(
    f"Validation dataset (test) contains {x_test.shape[1]} rows (5mC sites) and {x_test.shape[0]} columns (samples).")
print(f"\n{y_test['Clinical Trial'].value_counts(dropna=False).to_string()}\n")


Discovery dataset (train) contains 310545 rows (5mC sites) and 1142 columns (samples)

AAML1031    520
AAML0531    508
AML05        64
AAML03P1     36
CCG2961      14

Validation dataset (test) contains 310545 rows (5mC sites) and 204 columns (samples).

AML02    162
AML08     42



## Batch Correction with pyCombat

- __pyCombat__: a Python tool for batch effects correction in high-throughput molecular data using empirical Bayes methods

- __Github__: [https://epigenelabs.github.io/pyComBat/](https://epigenelabs.github.io/pyComBat/)

- __Implementation Paper__: [bioRxiv](https://doi.org/10.1101/2020.03.17.995431)

- __Original Paper__: [Biostatistics](https://pubmed.ncbi.nlm.nih.gov/16632515/)

In [10]:
from combat.pycombat import pycombat

# Correct batch effects in the training dataset
x_train2 = pycombat(x_train.T, y_train['Batch']).T

print('Succesfully corrected batch effects in the training dataset.')


Found 4 batches.
Adjusting for 0 covariate(s) or covariate level(s).
Standardizing Data across genes.
Fitting L/S model and finding priors.
Finding parametric adjustments.
Adjusting the Data
Succesfully corrected batch effects in the training dataset.


In [11]:
y_train = y_train[~y_train['Clinical Trial'].isin(['AML05'])]
x_train3 = x_train2.loc[y_train.index]


print(
    f"Discovery dataset (train) contains {x_train3.shape[1]} rows (5mC sites) and {x_train3.shape[0]} columns (samples)")
print(
    f"\n{y_train['Clinical Trial'].value_counts(dropna=False).to_string()}\n")
print(
    f"Validation dataset (test) contains {x_test.shape[1]} rows (5mC sites) and {x_test.shape[0]} columns (samples).")
print(f"\n{y_test['Clinical Trial'].value_counts(dropna=False).to_string()}\n")

Discovery dataset (train) contains 310545 rows (5mC sites) and 1078 columns (samples)

AAML1031    520
AAML0531    508
AAML03P1     36
CCG2961      14

Validation dataset (test) contains 310545 rows (5mC sites) and 204 columns (samples).

AML02    162
AML08     42



In [12]:
x_train3.to_pickle(output_path+'x_train.pkl') # Save methyl data
y_train.to_csv(output_path+'y_train.csv') # Save clinical data

print(
    f'Successfuly saved methyl data in x.pkl and clinical data in y.csv.\nPath: {output_path}')

Successfuly saved methyl data in x.pkl and clinical data in y.csv.
Path: ../Data/Processed_Data/


### Save Control and Relapse Data Separately

In [13]:
controls = df_[df_['Sample Type'].isin(['Bone Marrow Normal'])]

relapse = df_[df_['Sample Type'].isin(['Relapse', 'Recurrent Blood Derived Cancer - Bone Marrow',
                                       'Recurrent Blood Derived Cancer - Peripheral Blood'])]

# Merge control and relapse samples
t = pd.concat([controls, relapse], axis=0, join='outer',
              names=['Control', 'Relapse'])

# Join clinical data with methyl data
t2 = df_methyl.join(t, how='right')

# Save merged control and relapse samples
t2.to_pickle(output_path+'control_relapse.pkl')

print(
    f'Successfuly saved {controls.shape[0]} control samples and {relapse.shape[0]} relapse samples.\nPath: {output_path}')


Successfuly saved 147 control samples and 248 relapse samples.
Path: ../Data/Processed_Data/


## Watermark

In [14]:
%load_ext watermark

In [15]:
# produce a list of the loaded modules
%watermark -v -p pandas

Python implementation: CPython
Python version       : 3.10.10
IPython version      : 8.3.0

pandas: 1.5.3

